#Ассоциативные правила

Целью поиска ассоциативных правил является нахождение связонностей данных.

Постановка задачи:

- Мы анализируем чеки (они же транзакции) покупателей супермаркета. В чеке может быть любой набор товаров.

- Нам надо соотнести товары так, чтобы покупательная способность магазина увеличилась

- например некоторые люди могут покупать шоколад вместе с чипсами, нам надо найти и соотнести такие товары и к примеру поставить на соответствующие ближайшие полки или закупить товары, которые более всего соотносятся

Пример: 
- при покупке хлеба часто покупают молоко
- с вином часто покупают фрукты

**Определение**

Ассоциативное правило - закономерность вида $A \rightarrow B$, где A и B - множества объектов в корзине.

Для того, чтобы можно было сравнивать правила между собой, вводится несколько простых метрик:

**1) Support правила(поддержка)** - доля данных, в которых присутсвтует и объект A, и объект B

$$supp(A \rightarrow B)=\frac{|A \cap B|}{|N|}$$

**2) Confidence правила(достоверность)** - доля данных с объектами B, среди объектов A

$$conf(A \rightarrow B)=\frac{supp(A \cap B)}{supp(A)}$$

**3) Lift правила** - насколько confidence правила A->B выше/ниже общей частоты объектов B

$$lift(A \rightarrow B)=\frac{conf(A \rightarrow B)}{supp(B)} = \frac{supp(A \cap B)}{supp(A) \cdot supp(B)}$$

Зачем нужен Lift: высокий confidence может объясняться просто большой популярностью продукта B, а вовсе не его хорошими дополняющими свойствами к продукту A. Метрика Lift этой особенности лишена.

Support трактуется как "устойчивость" зависимости (как часто имеем возможность подобное правило наблюдать)

Confidence трактуется как "сила" зависимости (при совпадении нужного набора продуктов как часто это правило действует)

**Общий алгоритм** поиска ассоциативных правил:

1) Найти комбинации часто встречающихся вместе продуктов (**frequent itemsets**)

2) Для отобранных множеств-кандидатов построить на них все возможные ассоциативные правила, посчитать их метрики (confidence, support), отсортировать и вывести список наилучших правил

В правиле $A \rightarrow B$:
- Левая часть (A) называется Условие (**Antecedent**)
- Правая часть (B) - называется Следствие (**Consequent**)

Есть несколько способов представления товаров в чеке:
- денормализованный "плотный" формат - делаем табличку с 1000 колонками (по числу товаров в магазине) и отмечаем те, которые клиент купил

|Transaction_id| item1 | item2 | item3 | item4 | item5 | item6 | item7 | item8 | item9 | item10 |
|---|---|---|---|---|---|---|---|---|---|---|
|trn_1| 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 |
|trn_2| 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 |
|trn_3| 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 |
|trn_4| 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
|trn_5| 0 | 1 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 |

- денормализованный "разреженный" формат (строка с названиями/идентификаторам продукта):

|Transaction_id| items_list |
|---|---|
|trn_1| {item3, item8} |
|trn_2| {item1, item3, item6, item9} |
|trn_3| {item2, item6, item8} |
|trn_4| {item1, item4} |
|trn_5| {item2, item6, item7} |

- нормализованный - одна запись таблицы это пара (trn_id, item)

|Transaction_id | item_id |
|---|---|
|trn_1| item3 |
|trn_1| item8 |
|trn_2| item1 |
|trn_2| item3 |
|trn_2| item6 |
|trn_2| item9 |
|trn_3| item2 |
| ... | ... |
|trn_5| item7 |


Каждая конректная реализация алгоритма работает со своим форматом входных данных (одним их перечисленных выше), поэтому при использовании сторонних библиотек необходимо предварительно смотреть документацию.

#Apriori

Алгоритм Apriori ищет ассоциативные правила и применяется по отношению к базам данных, содержащим огромное количество транзакций.

Используем алгоритм на примере данных с чеками покупок в магазине

Загружаем библиотеки

In [205]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Загружаем сами данные

In [206]:
df = pd.read_csv("Market_Basket_Optimisation.csv")
df.head(5)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


В данных присутствуют различные продукты записанные в чеке, не купленные помечены как NaN для упращения работы их нужно заменить на первые попавшиеся значения

In [207]:
df.ffill(axis=1, inplace=True)
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs
2,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney
3,turkey,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado
4,mineral water,milk,energy bar,whole wheat rice,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea


Приведем транзакции в матричный вид

In [208]:
transactions = []
for i in range(0, 7501): 
    transactions.append([str(df.values[i,j]) for j in range(0, 20)])
transactions[:1]

[['shrimp',
  'almonds',
  'avocado',
  'vegetables mix',
  'green grapes',
  'whole weat flour',
  'yams',
  'cottage cheese',
  'energy drink',
  'tomato juice',
  'low fat yogurt',
  'green tea',
  'honey',
  'salad',
  'mineral water',
  'salmon',
  'antioxydant juice',
  'frozen smoothie',
  'spinach',
  'olive oil']]

Переведем транзакции для алгоритма в правильный вид

In [209]:
te = TransactionEncoder()
te.fit(transactions)

print("Список товаров, которые нашел TransactionEncoder = {}".format(te.columns_))

Список товаров, которые нашел TransactionEncoder = ['almonds', 'antioxydant juice', 'asparagus', 'avocado', 'babies food', 'bacon', 'barbecue sauce', 'black tea', 'blueberries', 'body spray', 'bramble', 'brownies', 'bug spray', 'burger sauce', 'burgers', 'butter', 'cake', 'candy bars', 'carrots', 'cauliflower', 'cereals', 'champagne', 'chicken', 'chili', 'chocolate', 'chocolate bread', 'chutney', 'cider', 'clothes accessories', 'cookies', 'cooking oil', 'corn', 'cottage cheese', 'cream', 'dessert wine', 'eggplant', 'eggs', 'energy bar', 'energy drink', 'escalope', 'extra dark chocolate', 'flax seed', 'french fries', 'french wine', 'fresh bread', 'fresh tuna', 'fromage blanc', 'frozen smoothie', 'frozen vegetables', 'gluten free bar', 'grated cheese', 'green beans', 'green grapes', 'green tea', 'ground beef', 'gums', 'ham', 'hand protein bar', 'herb & pepper', 'honey', 'hot dogs', 'ketchup', 'light cream', 'light mayo', 'low fat yogurt', 'magazines', 'mashed potato', 'mayonnaise', 'meat

Заполним полученные данные логическим типом true есть в чеке, false нет

In [210]:
transactions_onehot = te.transform(transactions)
transactions_onehot

array([[ True,  True, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False]])

Переведём данные обратно в датафрейм

In [211]:
df = pd.DataFrame(transactions_onehot, columns=te.columns_)
df.head(3)

,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,body spray,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,True,True,False,True,False,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Загрузим в apriori

- min_support -- минимальный support для правил (dtype = float).

In [212]:
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)
frequent_itemsets.head(5)

,support,itemsets
0,0.087188,(burgers)
1,0.081056,(cake)
2,0.059992,(chicken)
3,0.163845,(chocolate)
4,0.080389,(cookies)


В нашем случае получилось найти больше половины сочетающихся элементов при значении минимального support 0.05

In [213]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.05)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
1,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
2,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158
3,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
4,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
5,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314


- antecedent_support - частота набора продуктов из левой части правила A
- consequent_support - частота набора продуктов из правой части правила B


Суть метрики Conviction - как соотносится популярность других товаров (не B) с частотой ситуации когда наше правило не срабатывает:
- Если conv = 1, то правило бесполезно. Правило ошибается столько же, сколько покупатели выбирают другие товары
- Если conv > 1, то правило хорошо выделяет закономерность
- Если conv < 1, то правило не просто плохо работает, а выделяет антипаттерн (что никогда не покупают вместе) и в этом случае можно также воспользоваться для поиска несовместимых предметов и продуктов

Расмотрим правило: (chocolate) => (mineral water).

Прочитаем характеристики данного правила:
- В 53% чеков встречаются шоколад и минералка (support=0.5266)
- 32% тех, кто купил яйца, также шоколад и минералку (confidence=0.321400)
- Купившие шоколад покупают минералку на 34% чаще, чем все остальные покупатели (lift=1.348332)
- Купившие шоколад покупают другие продукты на 12% реже, чем просто покупающие другие продукты

Кроме apriori используются и другие алгоритмы:
- Eclat algorithm
- FP-Growth algorithm
- One-attribute-rule
- Zero-attribute-rule

#FP Деревья

- FP-деревья также является алгоритмом для поиска ассоциативных правил.
- Он является оптимизацией Apriori

Устанавливаем библиотеки

In [217]:
!pip install pyfpgrowth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [218]:
import pyfpgrowth

- найдём все правила, которые встречаются не менее 5 раз
- найдём все следствия из правил, которые происходят не реже чем в 50% случаев

Предупреждение: для данных такого размера алгоритм будет работать долго

Из-за этого ограничимся 50 элементами

In [228]:
patterns = pyfpgrowth.find_frequent_patterns(transactions[:50], 5)
rules = pyfpgrowth.generate_association_rules(patterns, 0.5)

In [229]:
patterns

{('avocado', 'avocado'): 171,
 ('avocado', 'avocado', 'avocado'): 969,
 ('avocado', 'avocado', 'turkey'): 171,
 ('avocado', 'eggs'): 17,
 ('avocado', 'eggs', 'eggs'): 136,
 ('avocado', 'fresh bread'): 15,
 ('avocado', 'fresh bread', 'fresh bread'): 105,
 ('avocado', 'fresh bread', 'fresh bread', 'spaghetti'): 105,
 ('avocado', 'fresh bread', 'spaghetti'): 15,
 ('avocado', 'green tea'): 14,
 ('avocado', 'green tea', 'green tea'): 78,
 ('avocado', 'muffins'): 19,
 ('avocado', 'muffins', 'muffins'): 171,
 ('avocado', 'toothpaste'): 13,
 ('avocado', 'toothpaste', 'toothpaste'): 78,
 ('avocado', 'turkey'): 19,
 ('bug spray', 'bug spray'): 171,
 ('bug spray', 'bug spray', 'bug spray'): 969,
 ('bug spray', 'bug spray', 'mineral water'): 171,
 ('bug spray', 'mineral water'): 19,
 ('chocolate', 'chocolate'): 153,
 ('chocolate', 'chocolate', 'chocolate'): 816,
 ('chocolate', 'eggs'): 19,
 ('chocolate', 'eggs', 'eggs'): 136,
 ('chocolate', 'eggs', 'french fries'): 16,
 ('chocolate', 'eggs', 'fren

Посмотрим теперь на ассоциативные правила, которые можно выделить среди таких паттернов

In [230]:
rules

{('avocado', 'avocado'): ((), 5.666666666666667),
 ('avocado', 'eggs'): ((), 8.0),
 ('avocado', 'fresh bread'): (('spaghetti',), 7.0),
 ('avocado', 'fresh bread', 'fresh bread'): (('spaghetti',), 1.0),
 ('avocado', 'fresh bread', 'spaghetti'): ((), 7.0),
 ('avocado', 'green tea'): ((), 5.571428571428571),
 ('avocado', 'muffins'): ((), 9.0),
 ('avocado', 'toothpaste'): ((), 6.0),
 ('avocado', 'turkey'): ((), 9.0),
 ('bug spray', 'bug spray'): ((), 5.666666666666667),
 ('bug spray', 'mineral water'): ((), 9.0),
 ('chocolate', 'chocolate'): ((), 5.333333333333333),
 ('chocolate', 'eggs'): ((), 7.157894736842105),
 ('chocolate', 'eggs', 'french fries'): ((), 7.5),
 ('chocolate', 'eggs', 'shampoo'): ((), 6.5),
 ('chocolate', 'french fries', 'french fries'): ((), 4.666666666666667),
 ('chocolate', 'low fat yogurt'): ((), 6.5),
 ('chocolate', 'shampoo'): (('eggs',), 6.5),
 ('chocolate', 'shampoo', 'shampoo'): (('eggs',), 1.0),
 ('chutney',): ((), 9.5),
 ('clothes accessories', 'clothes access

##FP-growth algorithm с другой библиотеки

In [231]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [243]:
# use fp-growth algorithm
from mlxtend.frequent_patterns.fpgrowth import fpgrowth

In [244]:
df.head(3)

,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,body spray,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,True,True,False,True,False,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [249]:
f_patterns = fpgrowth(df, min_support=0.05)
f_patterns.head(5)

,support,itemsets
0,0.238368,(71)
1,0.132116,(53)
2,0.076523,(64)
3,0.071457,(96)
4,0.065858,(80)


#One-Rule Algorithm

- Этот метод используется в алгоритме последовательного обучения для изучения правил.
- Он возвращает одно правило, которое охватывает по крайней мере некоторые примеры (как показано на рисунке 1).
- Тем не менее, что делает его действительно мощным, так это его способность создавать отношения между заданными атрибутами, следовательно, охватывая большее пространство гипотез.

Ниже приведены исходные данные Iris на них мы будем проверять работу алгоритма:

In [250]:
from mlxtend.data import iris_data
X, y = iris_data()

Ниже приведен дискретизированный набор данных. Каждый объект разделен на 4 квартиля.

In [251]:
import numpy as np


def get_feature_quartiles(X):
    X_discretized = X.copy()
    for col in range(X.shape[1]):
        for q, class_label in zip([1.0, 0.75, 0.5, 0.25], [3, 2, 1, 0]):
            threshold = np.quantile(X[:, col], q=q)
            X_discretized[X[:, col] <= threshold, col] = class_label
    return X_discretized.astype(np.int)

Xd = get_feature_quartiles(X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


Учитывая набор данных с категориальными признаками, мы можем использовать классификатор OneR, похожий на оценщик scikit-learn для классификации. Во-первых, давайте разделим набор данных на обучающие и тестовые данные:

In [252]:
from sklearn.model_selection import train_test_split
Xd_train, Xd_test, y_train, y_test = train_test_split(Xd, y, random_state=0, stratify=y)

In [253]:
%cd /content/Facewap

# Install the python version
!apt-get install python3.7

# Select the version
#!python setup.py
!python --version

/content/Facewap
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.7 is already the newest version (3.7.13-1+bionic3).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Python 3.7.13


In [254]:
%pip install mlxtend --upgrade
import mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [255]:
from mlxtend.classifier.oner import OneRClassifier

Далее мы можем обучить модель на обучающем наборе с помощью метода:OneRClassifierfit

In [256]:
oner = OneRClassifier()
oner.fit(Xd_train, y_train);

Индекс столбца выбранного объекта доступен через атрибут после подгонки модели:feature_idx_

In [257]:
oner.feature_idx_

2

Также доступна после примерка модели. В нем перечисляется общая ошибка для выбранного компонента (т.е. компонента, указанного в разделе ). Также в нем предусмотрены правила классификации:prediction_dict_feature_idx_

In [258]:
oner.prediction_dict_

{'rules (value: class)': {0: 0, 1: 1, 2: 1, 3: 2}, 'total error': 16}

Т.е. означает, что для выбранного признака (длина лепестка) существует 3 правила: - если значение равно 0, классифицировать как 0 (Iris-setosa) - если значение равно 1, классифицировать как 1 (Iris-versicolor) - если значение равно 2, классифицировать как 1 (Iris-versicolor) - если значение равно 3, классифицировать как 2 (Iris-virginica)'rules (value: class)': {0: 0, 1: 1, 2: 1, 3: 2}

После подгонки модели мы можем использовать объект для прогнозирования:oner

In [259]:
oner.predict(Xd_train)

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 2, 2, 1,
       0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 0, 0, 1, 2, 1, 1, 2, 2, 1, 0, 1,
       1, 1, 2, 0, 1, 2, 1, 2, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 1, 1,
       0, 1, 2, 1, 2, 0, 1, 2, 1, 1, 2, 0, 1, 0, 0, 1, 1, 2, 0, 0, 0, 1,
       0, 1, 2, 2, 2, 0, 1, 0, 2, 0, 1, 1, 1, 1, 0, 2, 2, 0, 1, 1, 0, 2,
       1, 2])

In [260]:
y_pred = oner.predict(Xd_train)
train_acc = np.mean(y_pred == y_train)  
print(f'Training accuracy {train_acc*100:.2f}%')

Training accuracy 85.71%


In [261]:
y_pred = oner.predict(Xd_test)
test_acc = np.mean(y_pred == y_test)  
print(f'Test accuracy {test_acc*100:.2f}%')

Test accuracy 84.21%


Вместо того, чтобы вычислять точность прогнозирования вручную, как показано выше, мы также можем использовать метод:score

In [262]:
test_acc = oner.score(Xd_test, y_test)
print(f'Test accuracy {test_acc*100:.2f}%')

Test accuracy 84.21%


#ECLAT

Алгоритм Эклата — алгоритм поиска глубины, основанный на пересечении множеств. Он подходит как для последовательного, так и для параллельного выполнения с свойствами, улучшающими локальность.

Устанавливаем библиотеки

In [263]:
!pip install pyECLAT
%cd /content/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content


Загружаем данные

In [264]:
df = pd.read_csv("Market_Basket_Optimisation.csv", header=None)

И собственно используем библиотеку

In [265]:
from pyECLAT import ECLAT
eclat_instance = ECLAT(data=df, verbose=True)

100%|██████████| 139/139 [00:00<00:00, 2629.17it/s]


После получения eclat_instance автоматически генерируется двоичный фрейм данных, среди других ресурсов, к которым можно получить доступ:

In [266]:
eclat_instance.df_bin

,5,11,tomatoes,oil,salt,cereals,frozen smoothie,hand protein bar,cream,gluten free bar,...,cake,2,fresh bread,babies food,mashed potato,whole weat flour,chocolate,green beans,olive oil,pancakes
0,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7500,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [267]:
eclat_instance.uniq_

[nan,
 '5',
 '11',
 'tomatoes',
 'oil',
 'salt',
 'cereals',
 'frozen smoothie',
 'hand protein bar',
 'cream',
 'gluten free bar',
 'salad',
 'mint',
 'eggplant',
 'pet food',
 'clothes accessories',
 'pepper',
 '16',
 'salmon',
 'magazines',
 'burgers',
 'grated cheese',
 'protein bar',
 '13',
 'honey',
 'low fat yogurt',
 'fromage blanc',
 '18',
 'avocado',
 'pasta',
 'yogurt cake',
 'shallot',
 'ground beef',
 'bacon',
 'mayonnaise',
 '6',
 'burger sauce',
 'bramble',
 'antioxydant juice',
 'spaghetti',
 'extra dark chocolate',
 'cookies',
 'pickles',
 '4',
 'sandwich',
 'tomato sauce',
 'shrimp',
 'oatmeal',
 'nonfat milk',
 'light cream',
 'whole wheat rice',
 'strawberries',
 'corn',
 'milk',
 'toothpaste',
 'zucchini',
 'cauliflower',
 '19',
 'cider',
 'black tea',
 'turkey',
 'herb & pepper',
 'vegetables mix',
 'chili',
 '8',
 'dessert wine',
 'spinach',
 'mushroom cream sauce',
 'frozen vegetables',
 'hot dogs',
 'almonds',
 'energy bar',
 'parmesan cheese',
 'gums',
 '7',
 

eclat_instance. поддержка, eclat_instance. подходит и eclat_instance. fit_all функции для выполнения вычислений. Пример:

In [268]:
get_ECLAT_indexes, get_ECLAT_supports = eclat_instance.fit(min_support=0.08,
                                                           min_combination=1,
                                                           max_combination=3,
                                                           separator=' & ',
                                                           verbose=True)

Combination 1 by 1


13it [00:00, 59.73it/s]


Combination 2 by 2


78it [00:00, 106.19it/s]


Combination 3 by 3


286it [00:02, 107.86it/s]


#SMPF

По сути, этот модуль вызывает инструмент командной строки Java SPMF, передает ему аргументы пользователя и анализирует выходные данные.
Кроме того, возможно преобразование данных в Pandas DataFrame и CSV.

Теоретически все алгоритмы, представленные в SPMF, можно вызывать. Ничто не жестко закодировано, нужный алгоритм и его параметры должны быть просмотрены в документации SPMF.

In [269]:
!pip install spmf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [270]:
from spmf import Spmf

spmf = Spmf("PrefixSpan", input_filename="Market_Basket_Optimisation.csv",
            output_filename="Market_Basket_Optimisation_output.csv", arguments=[0.7, 5])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output.csv")

>/content/spmf.jar
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 4 ms
 Frequent sequences count : 0
 Max memory (mb) : 0.0
 minsup = 1 sequences.
 Pattern count : 0


Empty DataFrame
Columns: []
Index: []


#Ссылки

https://habr.com/ru/post/66016/?ysclid=l3qfmutiwm


https://en.wikipedia.org/wiki/Association_rule_learning

https://www.geeksforgeeks.org/learn-one-rule-algorithm/

http://rasbt.github.io/mlxtend/user_guide/classifier/OneRClassifier/

https://habr.com/ru/company/ods/blog/353502/?ysclid=l3qjtxs4vv

https://pypi.org/project/pyECLAT/

https://loginom.ru/blog/fpg?ysclid=l3qfmjmhn8

https://pypi.org/project/spmf/

http://www.philippe-fournier-viger.com/spmf/index.php?link=documentation.php